<a href="https://colab.research.google.com/github/RahulManavalan/Machine-Learning/blob/master/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install tensorflow==1.0
import pandas as pd 
import numpy as np 

In [0]:
def read_goog_sp500_data():
  googFile = "/content/GOOG.csv"
  spFile = "/content/^IXIC.csv"

  goog = pd.read_csv(googFile,sep=",",usecols=[0,5],names=["Date","Goog"],header=0)
  sp = pd.read_csv(spFile,sep=",",usecols=[0,5],names=["Date","SP500"],header=0)


  goog["SP500"] = sp["SP500"]

  goog["Date"] = pd.to_datetime(goog["Date"],format="%Y-%m-%d")
  goog = goog.sort_values(["Date"],ascending=[True])

  returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ["float64","int64"]]].pct_change()

  return returns 

In [0]:
def read_goog_sp500_logistic_data():
  returns = read_goog_sp500_data() 
  returns["Intercept"] = 1 
  xData = np.array(returns[["SP500","Intercept"]][1:-1])
  yData = (returns["Goog"] > 0)[1:-1] 
  return (xData , yData) 

In [0]:
import tensorflow as tf 

xData , yData = read_goog_sp500_logistic_data() 

W = tf.Variable(tf.ones([1,2]) , name = "W") 
b = tf.Variable(tf.zeros([2]), name = "b")

x = tf.placeholder(tf.float32 , [None,1] , name="x")
y_ = tf.placeholder(tf.float32 , [None,2] , name="y_")

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_ , logits = y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


In [0]:
all_xs = np.expand_dims(xData[:,0],axis=1) 
all_ys = np.array([([1,0] if yEl == True else [0,1]) for yEl in yData])

In [0]:
dataset_size = len(all_xs)

In [0]:
def trainWithMultiplePointsperEpoch(steps,train,batch_size): 
  init = tf.global_variables_initializer() 

  with tf.Session() as sess: 
    sess.run(init) 

    for i in range(steps): 
      if dataset_size == batch_size: 
        batch_start_idx = 0 
      elif dataset_size < batch_size: 
        raise ValueError("The batch size must be smaller than the size of the dataset")
      else : 
        batch_start_idx = (i * batch_size) % (dataset_size)

      batch_end_idx = batch_start_idx + batch_size 

      batch_xs = all_xs[batch_start_idx : batch_end_idx] 
      batch_ys = all_ys[batch_start_idx : batch_end_idx]

      feed = {x : batch_xs , y_ : batch_ys} 
      sess.run(train, feed_dict = feed)

      if (i+1)%1000 == 0 : 
        print("After %d iterations:" %i) 
        print(sess.run(W))
        print(sess.run(b))

        print("Cross Entropy : %f" %sess.run(cross_entropy , feed_dict = feed))

    correct_prediction = tf.equal(tf.argmax(y_,1) , tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
    print("Accuracy: %f " %sess.run(accuracy , feed_dict = {x: all_xs , y_:all_ys} ))

In [0]:
trainWithMultiplePointsperEpoch(20000,train_step,dataset_size)